# 🎄 Santa 2025 - Advanced Optimizer V3

## Key Techniques:
1. **Global Rotation Optimization** - Rotates entire configurations to minimize bounding box (from best public notebook)
2. **Systematic bbox3 grid search** - n: 100-500, r: 10-50
3. **Simulated Annealing** - Escape local optima
4. **Boundary Tree Optimization** - Focus on trees that define bounding box
5. **Overlap auto-fix** - Reverts to backup if overlaps detected


In [ ]:
DEBUG = False 

MAX_HOURS = 0.05 if DEBUG else 8.5
SA_ITERATIONS = 50 if DEBUG else 300
BOUNDARY_ITERATIONS = 30 if DEBUG else 100
BBOX3_TIMEOUT = 120 if DEBUG else 600

print(f"🎄 Santa 2025 Optimizer V3")
print(f"   DEBUG mode: {DEBUG}")
print(f"   Max runtime: {MAX_HOURS} hours")

In [ ]:
import subprocess
import shutil
import os
import time
import math
import pandas as pd
import numpy as np
from numba import njit
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from shapely.ops import unary_union
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import random
from datetime import datetime, timedelta

# --- Setup Paths ---
INPUT_SUB = '/kaggle/input/santa-submission/submission.csv'
INPUT_BIN = '/kaggle/input/santa-submission/bbox3'
WORKING_DIR = '/kaggle/working/'

print("📂 Setting up environment...")

if os.path.exists(INPUT_SUB):
    shutil.copy(INPUT_SUB, os.path.join(WORKING_DIR, 'submission.csv'))
    shutil.copy(INPUT_SUB, os.path.join(WORKING_DIR, 'backup.csv'))  # Backup for overlap recovery
    print(f"✅ Copied submission.csv and backup.csv")

if os.path.exists(INPUT_BIN):
    shutil.copy(INPUT_BIN, os.path.join(WORKING_DIR, 'bbox3'))
    os.chmod('./bbox3', 0o755)
    print(f"✅ Copied and set permissions for bbox3")

getcontext().prec = 30
scale_factor = Decimal("1")

In [ ]:
# ============================================================
# NUMBA-OPTIMIZED CORE FUNCTIONS
# ============================================================

@njit(cache=True)
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x = np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2], np.float64)
    y = np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1], np.float64)
    return x, y

@njit(cache=True)
def score_group_fast(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c*tx[j] - s*ty[j] + xi
            Y = s*tx[j] + c*ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

@njit(cache=True)
def find_boundary_trees(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    min_x_tree = 0; min_y_tree = 0; max_x_tree = 0; max_y_tree = 0
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c*tx[j] - s*ty[j] + xi
            Y = s*tx[j] + c*ty[j] + yi
            if X < mnx: mnx = X; min_x_tree = i
            if X > mxx: mxx = X; max_x_tree = i
            if Y < mny: mny = Y; min_y_tree = i
            if Y > mxy: mxy = Y; max_y_tree = i
    return min_x_tree, min_y_tree, max_x_tree, max_y_tree

In [ ]:
# ============================================================
# SHAPELY-BASED TREE AND OVERLAP FUNCTIONS
# ============================================================

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        
        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal("0.0") * scale_factor), float(tip_y * scale_factor)),
            (float(top_w / Decimal("2") * scale_factor), float(tier_1_y * scale_factor)),
            (float(top_w / Decimal("4") * scale_factor), float(tier_1_y * scale_factor)),
            (float(mid_w / Decimal("2") * scale_factor), float(tier_2_y * scale_factor)),
            (float(mid_w / Decimal("4") * scale_factor), float(tier_2_y * scale_factor)),
            (float(base_w / Decimal("2") * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal("2") * scale_factor), float(base_y * scale_factor)),
            (float(trunk_w / Decimal("2") * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal("2")) * scale_factor), float(trunk_bottom_y * scale_factor)),
            (float(-(trunk_w / Decimal("2")) * scale_factor), float(base_y * scale_factor)),
            (float(-(base_w / Decimal("2")) * scale_factor), float(base_y * scale_factor)),
            (float(-(mid_w / Decimal("4")) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(mid_w / Decimal("2")) * scale_factor), float(tier_2_y * scale_factor)),
            (float(-(top_w / Decimal("4")) * scale_factor), float(tier_1_y * scale_factor)),
            (float(-(top_w / Decimal("2")) * scale_factor), float(tier_1_y * scale_factor)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated, 
            xoff=float(self.center_x * scale_factor), 
            yoff=float(self.center_y * scale_factor)
        )
    
    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

def create_trees_from_arrays(xs, ys, degs):
    return [ChristmasTree(str(xs[i]), str(ys[i]), str(degs[i])) for i in range(len(xs))]

def has_overlap_arrays(xs, ys, degs):
    if len(xs) <= 1:
        return False
    trees = create_trees_from_arrays(xs, ys, degs)
    return has_overlap(trees)

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        for idx in tree_index.query(poly):
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        trees.append(ChristmasTree(row["x"][1:], row["y"][1:], row["deg"][1:]))
    return trees

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1])))

def get_score_from_trees(trees, n):
    side = get_tree_list_side_length(trees)
    return float(side ** 2 / Decimal(n))

def eval_df_sub(df, verbose=False):
    failed = []
    total_score = 0.0
    scores = {}
    for n in range(1, 201):
        trees = load_configuration_from_df(n, df)
        score = get_score_from_trees(trees, n)
        scores[n] = score
        total_score += score
        if verbose:
            print(f"{n:3}  {score:.6f}")
        if has_overlap(trees):
            failed.append(n)
    
    if not failed:
        print("✅ No overlaps")
    else:
        print(f"❌ Overlaps in: {failed}")
    print(f"📊 Score: {total_score:.12f}")
    return total_score, scores, failed

In [ ]:
# ============================================================
# KEY TECHNIQUE: GLOBAL ROTATION OPTIMIZATION
# From the best public notebook - rotates ENTIRE configuration!
# ============================================================

def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length when points are rotated by angle_deg."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_global_rotation(trees):
    """
    Find the optimal rotation angle for the ENTIRE configuration.
    This rotates all trees together to minimize the bounding box.
    """
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points)
    
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        hull_points = points_np
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0.001, 89.999),
        method='bounded'
    )
    
    found_angle_deg = res.x
    found_side = res.fun
    
    improvement = initial_side - found_side
    
    if improvement > 1e-8:
        return Decimal(str(found_side)), found_angle_deg
    else:
        return Decimal(str(initial_side)), 0.0

def apply_global_rotation(trees, angle_deg):
    """Apply rotation to entire configuration around its center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]
    
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    rotated_trees = []
    for i in range(len(trees)):
        new_angle = float(trees[i].angle) + angle_deg
        new_tree = ChristmasTree(str(rotated[i, 0]), str(rotated[i, 1]), str(new_angle))
        rotated_trees.append(new_tree)
    
    return rotated_trees

def fix_direction(df):
    """
    Apply global rotation optimization to all configurations.
    This is the KEY technique from the best public notebook!
    """
    improved_count = 0
    total_improvement = 0.0
    
    for n in range(1, 201):
        trees = load_configuration_from_df(n, df)
        if len(trees) <= 1:
            continue
        
        current_side = get_tree_list_side_length(trees)
        current_score = float(current_side ** 2 / Decimal(n))
        
        best_side, best_angle = optimize_global_rotation(trees)
        
        if best_side < current_side - Decimal("1e-10"):
            rotated_trees = apply_global_rotation(trees, best_angle)
            
            if not has_overlap(rotated_trees):
                new_side = get_tree_list_side_length(rotated_trees)
                new_score = float(new_side ** 2 / Decimal(n))
                
                if new_score < current_score - 1e-15:
                    group_mask = df["id"].str.startswith(f"{n:03d}_")
                    indices = df[group_mask].index
                    
                    for i, idx in enumerate(indices):
                        df.at[idx, 'x'] = f"s{rotated_trees[i].center_x}"
                        df.at[idx, 'y'] = f"s{rotated_trees[i].center_y}"
                        df.at[idx, 'deg'] = f"s{rotated_trees[i].angle}"
                    
                    improved_count += 1
                    total_improvement += current_score - new_score
    
    return improved_count, total_improvement

print("✅ Global rotation optimization loaded")

In [ ]:
# ============================================================
# LOCAL OPTIMIZATION STRATEGIES (with overlap checking)
# ============================================================

def simulated_annealing_config(df, config_n, max_iterations=300):
    tx, ty = make_polygon_template()
    group_mask = df["id"].str.startswith(f"{config_n:03d}_")
    group_data = df[group_mask].copy()
    
    xs = np.array([float(v[1:]) for v in group_data["x"]])
    ys = np.array([float(v[1:]) for v in group_data["y"]])
    degs = np.array([float(v[1:]) for v in group_data["deg"]])
    
    n_trees = len(xs)
    if n_trees <= 1:
        return False, 0
    
    original_score = score_group_fast(xs, ys, degs, tx, ty)
    current_score = original_score
    best_score = original_score
    best_xs, best_ys, best_degs = xs.copy(), ys.copy(), degs.copy()
    
    temperature = 0.001
    cooling_rate = 0.995
    step_xy = 0.002 / np.sqrt(n_trees)
    step_rot = 0.5 / np.sqrt(n_trees)
    
    for _ in range(max_iterations):
        tree_idx = random.randint(0, n_trees - 1)
        new_xs, new_ys, new_degs = xs.copy(), ys.copy(), degs.copy()
        
        move = random.choice(['translate', 'rotate', 'both'])
        if move in ['translate', 'both']:
            new_xs[tree_idx] += random.gauss(0, step_xy)
            new_ys[tree_idx] += random.gauss(0, step_xy)
        if move in ['rotate', 'both']:
            new_degs[tree_idx] += random.gauss(0, step_rot)
        
        new_score = score_group_fast(new_xs, new_ys, new_degs, tx, ty)
        delta = new_score - current_score
        
        if delta < 0 or random.random() < math.exp(-delta / temperature):
            if not has_overlap_arrays(new_xs, new_ys, new_degs):
                xs, ys, degs = new_xs, new_ys, new_degs
                current_score = new_score
                if current_score < best_score:
                    best_score = current_score
                    best_xs, best_ys, best_degs = xs.copy(), ys.copy(), degs.copy()
        
        temperature *= cooling_rate
    
    improved = best_score < original_score - 1e-15
    if improved and not has_overlap_arrays(best_xs, best_ys, best_degs):
        indices = df[group_mask].index
        for i, idx in enumerate(indices):
            df.at[idx, 'x'] = f"s{best_xs[i]}"
            df.at[idx, 'y'] = f"s{best_ys[i]}"
            df.at[idx, 'deg'] = f"s{best_degs[i]}"
        return True, original_score - best_score
    return False, 0

def optimize_boundary_trees(df, config_n, iterations=100):
    tx, ty = make_polygon_template()
    group_mask = df["id"].str.startswith(f"{config_n:03d}_")
    group_data = df[group_mask].copy()
    
    xs = np.array([float(v[1:]) for v in group_data["x"]])
    ys = np.array([float(v[1:]) for v in group_data["y"]])
    degs = np.array([float(v[1:]) for v in group_data["deg"]])
    
    n_trees = len(xs)
    if n_trees <= 1:
        return False, 0
    
    original_score = score_group_fast(xs, ys, degs, tx, ty)
    best_score = original_score
    best_xs, best_ys, best_degs = xs.copy(), ys.copy(), degs.copy()
    
    step = 0.0005
    rot_step = 0.2
    
    for _ in range(iterations):
        boundary_trees = set(find_boundary_trees(xs, ys, degs, tx, ty))
        for tree_idx in boundary_trees:
            cx, cy = np.mean(xs), np.mean(ys)
            dx, dy = cx - xs[tree_idx], cy - ys[tree_idx]
            norm = np.sqrt(dx*dx + dy*dy)
            if norm > 1e-10:
                dx, dy = dx/norm * step, dy/norm * step
            
            new_xs, new_ys = xs.copy(), ys.copy()
            new_xs[tree_idx] += dx
            new_ys[tree_idx] += dy
            new_score = score_group_fast(new_xs, new_ys, degs, tx, ty)
            if new_score < best_score and not has_overlap_arrays(new_xs, new_ys, degs):
                best_score = new_score
                xs, ys = new_xs, new_ys
                best_xs, best_ys = xs.copy(), ys.copy()
            
            for drot in [-rot_step, rot_step]:
                new_degs = degs.copy()
                new_degs[tree_idx] += drot
                new_score = score_group_fast(xs, ys, new_degs, tx, ty)
                if new_score < best_score and not has_overlap_arrays(xs, ys, new_degs):
                    best_score = new_score
                    degs = new_degs
                    best_degs = degs.copy()
    
    improved = best_score < original_score - 1e-15
    if improved and not has_overlap_arrays(best_xs, best_ys, best_degs):
        indices = df[group_mask].index
        for i, idx in enumerate(indices):
            df.at[idx, 'x'] = f"s{best_xs[i]}"
            df.at[idx, 'y'] = f"s{best_ys[i]}"
            df.at[idx, 'deg'] = f"s{best_degs[i]}"
        return True, original_score - best_score
    return False, 0

print("✅ Local optimizers loaded")

In [ ]:
# ============================================================
# OVERLAP AUTO-FIX FROM BACKUP
# ============================================================

def fix_overlaps_from_backup(df, backup_df, failed_configs):
    """Replace overlapping configs with versions from backup."""
    for n in failed_configs:
        group_id = f"{n:03d}"
        
        backup_mask = backup_df["id"].str.startswith(f"{group_id}_")
        backup_group = backup_df[backup_mask]
        
        if len(backup_group) == 0:
            print(f"  ⚠️ No backup for config {n}")
            continue
        
        main_mask = df["id"].str.startswith(f"{group_id}_")
        main_indices = df[main_mask].index
        backup_indices = backup_group.index
        
        for main_idx, backup_idx in zip(main_indices, backup_indices):
            df.at[main_idx, 'x'] = backup_df.at[backup_idx, 'x']
            df.at[main_idx, 'y'] = backup_df.at[backup_idx, 'y']
            df.at[main_idx, 'deg'] = backup_df.at[backup_idx, 'deg']
        
        print(f"  ✅ Fixed config {n} from backup")

print("✅ Overlap fix utility loaded")

In [ ]:
# ============================================================
# MAIN OPTIMIZATION LOOP
# ============================================================

def main():
    start_time = time.time()
    timeout = timedelta(hours=MAX_HOURS)
    
    # Load initial submission
    df = pd.read_csv("submission.csv")
    backup_df = pd.read_csv("backup.csv")
    
    initial_score, initial_scores, failed = eval_df_sub(df)
    
    if failed:
        print(f"⚠️ Initial overlaps in: {failed}")
        fix_overlaps_from_backup(df, backup_df, failed)
        df.to_csv("submission.csv", index=False)
        initial_score, initial_scores, _ = eval_df_sub(df)
    
    best_score = initial_score
    best_df = df.copy()
    
    # Parameter grid for bbox3
    n_values = [100, 150, 200, 300, 400, 500] if not DEBUG else [100, 200]
    r_values = [10, 20, 30, 40, 50] if not DEBUG else [20, 30]
    param_grid = [(n, r) for r in r_values for n in n_values]
    param_idx = 0
    
    print(f"\n{'='*60}")
    print(f"🎄 SANTA 2025 OPTIMIZER V3")
    print(f"   Initial score: {initial_score:.12f}")
    print(f"   Max runtime: {MAX_HOURS} hours")
    print(f"   Parameter grid: {len(param_grid)} combinations")
    print(f"{'='*60}")
    
    cycle = 0
    total_improvements = 0
    
    while True:
        # Check time
        elapsed = timedelta(seconds=time.time() - start_time)
        if elapsed > timeout:
            print(f"\n⏰ Time limit reached!")
            break
        
        cycle += 1
        print(f"\n--- Cycle {cycle} (elapsed: {elapsed}) ---")
        
        # Phase 1: Global rotation optimization (KEY TECHNIQUE!)
        print("  [Phase 1] Global rotation optimization")
        improved_count, rot_improvement = fix_direction(df)
        if improved_count > 0:
            print(f"    ✅ Rotated {improved_count} configs, gain: {rot_improvement:.12f}")
            total_improvements += improved_count
        
        # Validate
        _, _, failed = eval_df_sub(df)
        if failed:
            print(f"    ⚠️ Rotation created overlaps, reverting...")
            fix_overlaps_from_backup(df, backup_df, failed)
        
        # Phase 2: bbox3 optimization
        if os.path.exists('./bbox3'):
            print("  [Phase 2] bbox3 optimization")
            
            n, r = param_grid[param_idx % len(param_grid)]
            param_idx += 1
            
            print(f"    Running bbox3 n={n}, r={r}")
            
            try:
                subprocess.run(
                    ["./bbox3", "-n", str(n), "-r", str(r)],
                    capture_output=True,
                    timeout=BBOX3_TIMEOUT
                )
            except subprocess.TimeoutExpired:
                print(f"    ⏱️ bbox3 timeout")
            except Exception as e:
                print(f"    ⚠️ bbox3 error: {e}")
            
            df = pd.read_csv("submission.csv")
            new_score, _, failed = eval_df_sub(df)
            
            if failed:
                print(f"    ❌ bbox3 created overlaps, reverting")
                fix_overlaps_from_backup(df, backup_df, failed)
                df.to_csv("submission.csv", index=False)
            elif new_score < best_score:
                improvement = best_score - new_score
                best_score = new_score
                best_df = df.copy()
                backup_df = df.copy()
                backup_df.to_csv("backup.csv", index=False)
                total_improvements += 1
                print(f"    ✅ bbox3 improved by {improvement:.12f}")
        
        # Phase 3: Local optimization
        print("  [Phase 3] Local optimization")
        _, current_scores, _ = eval_df_sub(df)
        sorted_configs = sorted(current_scores.items(), key=lambda x: x[1], reverse=True)
        worst_configs = [c[0] for c in sorted_configs[:30]]
        
        configs_to_try = min(5 if DEBUG else 10, len(worst_configs))
        for config_n in random.sample(worst_configs, configs_to_try):
            elapsed = timedelta(seconds=time.time() - start_time)
            if elapsed > timeout:
                break
            
            strategy = random.choice(['SA', 'Boundary'])
            try:
                if strategy == 'SA':
                    improved, gain = simulated_annealing_config(df, config_n, SA_ITERATIONS)
                else:
                    improved, gain = optimize_boundary_trees(df, config_n, BOUNDARY_ITERATIONS)
                
                if improved:
                    total_improvements += 1
                    print(f"    ✅ {strategy} config {config_n}: {gain:.12f}")
            except Exception as e:
                pass
        
        # Save progress
        df.to_csv("submission.csv", index=False)
        new_score, _, failed = eval_df_sub(df)
        
        if failed:
            print(f"  ❌ Overlaps detected, reverting")
            df = best_df.copy()
            df.to_csv("submission.csv", index=False)
        elif new_score < best_score:
            best_score = new_score
            best_df = df.copy()
            backup_df = df.copy()
            backup_df.to_csv("backup.csv", index=False)
            print(f"  📈 New best: {best_score:.12f}")
        
        if cycle % 5 == 0:
            print(f"\n📊 Status: Score={best_score:.12f}, Improvements={total_improvements}")
        
        if DEBUG and cycle >= 2:
            print("\n🔧 DEBUG mode: stopping after 2 cycles")
            break
    
    # Final save
    best_df.to_csv("submission.csv", index=False)
    
    elapsed = timedelta(seconds=time.time() - start_time)
    
    print(f"\n{'='*60}")
    print("OPTIMIZATION COMPLETE")
    print(f"{'='*60}")
    final_score, _, _ = eval_df_sub(best_df)
    
    print(f"\n📈 Results:")
    print(f"   Initial:     {initial_score:.12f}")
    print(f"   Final:       {final_score:.12f}")
    print(f"   Improvement: {initial_score - final_score:.12f}")
    print(f"   Total improvements: {total_improvements}")
    print(f"   Total time: {elapsed}")
    print(f"   Cycles: {cycle}")
    
    if DEBUG:
        print(f"\n💡 For full optimization, set DEBUG = False")

main()

In [ ]:
# Final validation
print("\n" + "="*60)
print("FINAL VALIDATION")
print("="*60)

df = pd.read_csv("submission.csv")
final_score, _, failed = eval_df_sub(df)

if failed:
    print(f"\n❌ Overlaps detected: {failed}")
    backup_df = pd.read_csv("backup.csv")
    fix_overlaps_from_backup(df, backup_df, failed)
    df.to_csv("submission.csv", index=False)
    print("\nAfter fix:")
    eval_df_sub(df)
else:
    print(f"\n✅ No overlaps - ready to submit!")